In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score

In [2]:
lgb_sub = pd.read_csv('prob/sub_lgb.csv')
xgb_sub = pd.read_csv('prob/sub_xgb.csv')
cbt_sub = pd.read_csv('prob/sub_cbt.csv')

In [3]:
lgb_oof = pd.read_csv('prob/oof_lgb.csv')
xgb_oof = pd.read_csv('prob/oof_xgb.csv')
cbt_oof = pd.read_csv('prob/oof_cbt.csv')

In [4]:
sub = lgb_sub.copy()
sub.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
sub['prob_xgb'] = xgb_sub['prob'].values.tolist()
sub['prob_cbt'] = cbt_sub['prob'].values.tolist()
sub.head()

,id,prob_lgb,prob_xgb,prob_cbt
0,800000,0.184450,0.052672,0.055286
1,800001,0.630223,0.310593,0.338298
2,800002,0.575915,0.266404,0.332441
3,800003,0.616928,0.299573,0.300399
4,800004,0.703135,0.363906,0.404484


In [5]:
oof = lgb_oof.copy()
oof.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
oof['prob_xgb'] = xgb_oof['prob'].values.tolist()
oof['prob_cbt'] = cbt_oof['prob'].values.tolist()
oof.head()

,id,prob_lgb,isDefault,prob_xgb,prob_cbt
0,1,0.547366,0.0,0.298526,0.231715
1,23,0.701218,1.0,0.341079,0.309012
2,28,0.361765,0.0,0.115245,0.127430
3,32,0.644193,0.0,0.278753,0.322757
4,42,0.632134,1.0,0.301049,0.299982


In [6]:
auc1 = roc_auc_score(oof['isDefault'], oof['prob_lgb'])
auc2 = roc_auc_score(oof['isDefault'], oof['prob_xgb'])
auc3 = roc_auc_score(oof['isDefault'], oof['prob_cbt'])

auc1, auc2, auc3

(0.7429533392804317, 0.7432809151596199, 0.7433005718659207)

In [7]:
w_lgb = 0.5
w_xgb = 0.5
w_cbt = 0.5

oof['prob'] = oof['prob_lgb'] ** w_lgb * oof['prob_xgb'] ** w_xgb * oof['prob_cbt'] ** w_cbt
auc = roc_auc_score(oof['isDefault'], oof['prob'])
auc

0.7442769399676221

In [8]:
sub['prob'] = sub['prob_lgb'] ** w_lgb * sub['prob_xgb'] ** w_xgb * sub['prob_cbt'] ** w_cbt
sub.head()

,id,prob_lgb,prob_xgb,prob_cbt,prob
0,800000,0.184450,0.052672,0.055286,0.023176
1,800001,0.630223,0.310593,0.338298,0.257331
2,800002,0.575915,0.266404,0.332441,0.225843
3,800003,0.616928,0.299573,0.300399,0.235623
4,800004,0.703135,0.363906,0.404484,0.321710


In [9]:
sub.rename({'prob': 'isDefault'}, axis=1, inplace=True)
sub[['id', 'isDefault']].to_csv('sub/ronghe.csv', index=False)